<a href="https://colab.research.google.com/github/MaleNurse/YOLOv5_Trainer/blob/main/YOLOv5_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup environment

Connect Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Update environment

In [2]:
#! wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
#! mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
#! apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
#! add-apt-repository "deb https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/ /"
#! rm /etc/apt/sources.list.d/cuda.list
! apt update
! apt upgrade 
#! apt install libcudnn8 libcudnn8-dev libnccl-dev libnccl2 --allow-change-held-packages
! echo "Segment complete"

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [49.8 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ge

Check GPU and Driver Version (req >=460)

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Mar 29 14:54:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Install YOLOv5 and depends

In [4]:
%cd /content
!git clone https://github.com/ultralytics/yolov5  # clone repo
#!pip install imgaug==0.2.5 torch==1.8.0 torchvision==0.8.1 numpy==1.19.2 pandas==1.1.0 # deal w/ dependency issues
!pip install -U -r yolov5/requirements.txt  # install dependencies 
#!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html #use if CUDA v11 available
!pip install wandb

!echo "Segment complete RESTART"
!echo "RESTART THE RUNTIME"
!echo "RESTART THE RUNTIME"
!echo "RESTART THE RUNTIME"
!echo "RESTART THE RUNTIME"
!echo "RESTART THE RUNTIME"

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 5598 (delta 4), reused 1 (delta 0), pack-reused 5581
Receiving objects: 100% (5598/5598), 8.44 MiB | 19.74 MiB/s, done.
Resolving deltas: 100% (3818/3818), done.
     |████████████████████████████████| 10.3MB 4.0MB/s 
     |████████████████████████████████| 15.3MB 213kB/s 
     |████████████████████████████████| 50.4MB 63kB/s 
     |████████████████████████████████| 2.2MB 32.2MB/s 
     |████████████████████████████████| 645kB 42.0MB/s 
     |████████████████████████████████| 27.4MB 111kB/s 
     |████████████████████████████████| 804.1MB 21kB/s 
     |████████████████████████████████| 17.4MB 160kB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
Requirement already up-to-date: tensorboard>=2.4.1 in /usr/local/lib/python3.7/dist-packages (from -r yolov5/requirements.txt (line 15)) (2.4.1)
Requiremen

     |████████████████████████████████| 2.0MB 4.2MB/s 
     |████████████████████████████████| 163kB 36.6MB/s 
     |████████████████████████████████| 133kB 39.4MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=f97e6e69fe72a106091879e0f3b80035f3d8fb8dbd8dc4872f26dae2d4bdcc13
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=2a2561a8af9355bde1257eca4ba830578bd72931dcf5061aa4b47362b2bd58e7
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32
Segment complete RESTART
RESTART THE RUNTIME
RESTART THE RUNTIME
RESTART THE RUNTIME
RESTART THE RUNTIME
RESTART THE RUNTIME


Update YOLOv5 if, for some reason, it's already cloned

In [ ]:
%cd /content/yolov5
!git pull

/content/yolov5
Already up to date.


Check Torch environment (ensure no errors thrown)

In [1]:
%cd /content/yolov5
import torch
from IPython.display import Image  # for displaying images
from utils.google_utils import gdrive_download  # for downloading models/datasets

print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

/content/yolov5
torch 1.8.1+cu102 _CudaDeviceProperties(name='Tesla K80', major=3, minor=7, total_memory=11441MB, multi_processor_count=13)


Customize iPython writefile so we can write variables

In [2]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))
!echo "Segment complete"

Segment complete


# Setup training model

Define training parameters and make dependant paths/strings

**Known good configs**

yolov5x batch 18 img 640 epoch 250 w/ deepstack mod

yolov5l batch 20 img 736 epoch ??? w/ deepstack mod

In [3]:
# Set variables above separator. Don't delete the quote marks!

# Known good configs
# yolov5x batch 18 img 640 epoch 250 
# yolov5l batch 20 img 736 epoch 200 w/ checkpoint

model_size = 'l'        # trailing end of the training model desired (s, m, l, or x)
check = 'y'             # use a checkpoint as a starting point in the training? ('y' if yes)
deepstack = 'y'         # training for deepstack server? ('y' is yes)
img_size = '736'        # to what resolution to scale training images (e.g. 640) must be mult of 32
batch_size = '12'
epochs = '75'
comment = 'a4u1e75'        # something to distringuish the run name, if needed (e.g. aug_dsx)
test_img_size = '736'   # to what resolution to scale testing images (e.g. 960)  must be mult of 32
test_conf = '0.5'       # confidence threshold for inference testing (e.g. 0.5)

##########################################################################################

import os.path, shutil, os
from os import path

i = 1
j = 1

#make base model path
model_yaml = '/content/yolov5/models/yolov5' + model_size + '.yaml'

if not comment: 
  #make run name if no comment entered
  run_name = '5' + model_size + '_i' + img_size + '_b' + batch_size
else:
  #make run name if comment exists
  run_name = '5' + model_size + '_i' + img_size + '_b' + batch_size + '_' + comment

if deepstack == 'y':
  run_name = run_name + '_ds'

#if directory exists with current run name, start incrementing run name by 1 until unique
if path.exists(path.join('/content/drive/MyDrive/colab_out/', run_name)):                 
  while path.exists(path.join('/content/drive/MyDrive/colab_out/', run_name + str(i))):
    i += 1
  run_name = run_name + str(i)

#set path to run
run_path = '/content/drive/MyDrive/colab_out/' + run_name

exp_dir = '/content/drive/MyDrive/colab_out/exp'

#find next available exp directory if "exp" already exists
if path.exists('/content/drive/MyDrive/colab_out/exp'):                 
  while path.exists(path.join('/content/drive/MyDrive/colab_out/exp' + str(j))):
    j += 1
  exp_dir = exp_dir + str(j)

#make string for inference testing output
exp_img = path.join(exp_dir, '*.jpg')

#set path to weights
weights = run_path + '/weights/best.pt'

#set checkpoint name
if check == 'y':
  chkpnt = 'yolov5' + model_size + '.pt'
  #if training for deepstack, copy v3 weight file to correct path so v4 isn't downloaded
  if deepstack == 'y':
    chkpnt_tmp = path.join('/content/drive/MyDrive/v3_weights', chkpnt)
    shutil.copy(chkpnt_tmp, '/content/yolov5/')
else:
  chkpnt = ''

!echo "This run is called $run_name and the exp folder will be $exp_dir"

This run is called 5l_i736_b12_a4u1e75_ds and the exp folder will be /content/drive/MyDrive/colab_out/exp


Define number of classes based on YAML from Roboflow

In [4]:
%cd /content/drive/MyDrive/my-dataset
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])
!echo "Segment complete"

/content/drive/MyDrive/my-dataset
Segment complete


Modify Roboflow YAML with correct directories

In [5]:
%cd /content/drive/MyDrive/my-dataset
!sed 's/\.\./\/content\/drive\/MyDrive\/my-dataset/' data.yaml >temp.yaml
!mv temp.yaml data.yaml
!echo "Segment complete"

/content/drive/MyDrive/my-dataset
Segment complete


Output desired model size (adjust as necessary)

In [6]:
%cat $model_yaml

# parameters
nc: 80  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3,

Create custom model using exact class size.

Copy the cat output above starting at depth_multiple and replace the same below

In [7]:
%%writetemplate /content/yolov5/models/custom_yolov5.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

Adjust model if training for DeepStack

In [8]:
%cd /content/yolov5/models/
!sed 's/C3/BottleneckCSP/' custom_yolov5.yaml >temp.yaml
!mv temp.yaml custom_yolov5.yaml
!echo "Segment complete"

/content/yolov5/models
Segment complete


Make sure the custom model looks OK (and exists)

In [9]:
!cat /content/yolov5/models/custom_yolov5.yaml


# parameters
nc: 11  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1

# Train!

In [ ]:
%%time
%cd /content/yolov5/
!python train.py --batch $batch_size --img-size $img_size --epochs $epochs --data '/content/drive/MyDrive/my-dataset/data.yaml' --cfg ./models/custom_yolov5.yaml --weights '$chkpnt' --name $run_name  --project /content/drive/MyDrive/colab_out/ --cache-images

/content/yolov5
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v4.0-171-g1b100cd torch 1.8.1+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=12, bbox_interval=-1, bucket='', cache_images=True, cfg='./models/custom_yolov5.yaml', data='/content/drive/MyDrive/my-dataset/data.yaml', device='', entity=None, epochs=75, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[736, 736], linear_lr=False, local_rank=-1, multi_scale=False, name='5l_i736_b12_a4u1e75_ds', noautoanchor=False, nosave=False, notest=False, project='/content/drive/MyDrive/colab_out/', quad=False, rect=False, resume=False, save_dir='/content/drive/MyDrive/colab_out/5l_i736_b12_a4u1e75_ds', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=12, upload_dataset=False, weights='yolov5l.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir /content/drive/MyDrive/col

# Evaluate training

Start tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $run_path

# Test completed weights

Inference speed test

In [ ]:
# when we ran this, we saw .007 second inference time. That is 140 FPS on a TESLA P100!
%cd /content/yolov5/
!python detect.py --weights $weights --img $test_img_size --conf $test_conf --source /content/drive/MyDrive/my-dataset/test/images --project /content/drive/MyDrive/colab_out/

Display inference on all test images

In [ ]:
import glob
import os.path
from os import path
from IPython.display import Image, display

for imageName in glob.glob(exp_img): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# Set variables if environment lost in a runtime disconnection

Make sure to rerun environment setup, too

In [ ]:
run_name = '5x_i640_b18_nopre_ds' # look for first two in /content/drive/MyDrive/colab_out/
exp_num = ''                   # leave as '' if just exp
test_img_size = '1280'           # to what resolution to scale testing images (e.g. 960) must be mult of 32
test_conf = '0.5'               # confidence threshold for inference testing (e.g. 0.5)

#############################################################################

import os.path
from os import path

run_path = '/content/drive/MyDrive/colab_out/' + run_name
weights = run_path + '/weights/best.pt'
exp_img = path.join('/content/drive/MyDrive/colab_out/exp' + exp_num, '*.jpg')